(introduction-to-simphony)=
# Introduction to simphony

Before we start this tutorial, you should know the basics of Python. We expect you to have an Python environment set up,
with the [``simphony``](https://pypi.org/project/simphony/) package installed.

Our goal with this tutorial is to define and simulate a simple circuit. In simphony, circuits are represented all in a
single Python file. We'll go through the typical objects found in every circuit definition, in order.

```{note}
Simphony uses SPICE-like concepts--such as components, ports, and nets--to define circuits. This should make simphony
intuitive for all those familiar with SPICE, which is commonly used to define electronic circuits. 
```

## Models

Models are the basic components in simphony, which are used to represent an element in a photonic circuit. Each model has a 

The base Model class provides a common interface for all models, including methods to get and set the scattering 
parameters, get references to ports, rename ports, and check if a model is connected to any others. It also 
provides methods to convert the model to a scikit-rf network, which can be useful for further analysis and 
simulation.


```{eval-rst}
Here's an overview of the Model parent class :py:class:`simphony.models.Model`:

.. autoclass:: simphony.models.Model
    :noindex:
```

```{note}
A basic model has no ``__init__()`` function. It is only required if the model takes in parameters (width or length, for
example) that  affect the scattering parameters.
```

All models in Simphony extend this parent class, but redefine ports, and s-parameters to match the device
they represent.

## Instantiating Models

Before we can use a model in our circuit, we need to instantiate it. When we instantiate a model we call the resulting
object a component. The difference between models and components is that we can add any kind of state to a component
after it has been instantiated, outside of what the model defines.

Simphony includes a default library of models from the [SiEPIC PDK](https://github.com/SiEPIC) (developed at the University of British Columbia).
We might instantiate a couple of models with the following (note in the docstring for the model that length should be defined in microns):

In [8]:
from simphony.libraries import siepic

# waveguide of 2.5 mm length
wg1 = siepic.Waveguide(length=2500, loss=3)
# waveguide of 7.5 mm length
wg2 = siepic.Waveguide(length=7500, loss=3)

```{eval-rst}
These are both :py:class:`~simphony.libraries.siepic.Waveguide` models. The SiEPIC components are parameterizable, so we can pass different parameters when instantiating them. In this case ``wg1`` will be a shorter waveguide than ``wg2``.
Thus the two will have differing s-parameters for each model since each waveguide was set to have 3 dB of loss per cm.
```

```{note}
The convention in simphony is to use microns for units of length.
```

## Creating a Circuit

```{eval-rst}
The :py:class:`~simphony.models.Port` class is used as an interface to connect two components in a circuit. As an end user,
you should rarely have to interact with ports directly; instead, there are :py:class:`~simphony.circuit.Circuit` methods that will handle
connecting ports for you. Let's give an example.
```

The simplest way to connect ports is as follows:

In [9]:
from simphony.circuit import Circuit

In [10]:
wg1 = siepic.Waveguide(length=2500, loss=3)
wg2 = siepic.Waveguide(length=7500, loss=3)

ckt = Circuit()
ckt.connect(wg1, wg2)

This will connect the first unconnected optical port on each component together. However, if we want the first port of ``wg1`` to
be an input, and instead connect its second port to ``wg2`` as an output, we have to connect the ports explicitly (recalling that ports are 0-indexed):

In [11]:
wg1 = siepic.Waveguide(length=2500, loss=3)
wg2 = siepic.Waveguide(length=7500, loss=3)

ckt = Circuit()
ckt.connect(wg1.o(1), wg2.o(0))

By default, a model instantiates its ports with names "o0", "o1", etc. Here we specify "o1" of ``wg1`` must
connect to "o0" of ``wg2``. 

In [12]:
wg1 = siepic.Waveguide(length=2500, loss=3)
wg2 = siepic.Waveguide(length=7500, loss=3)

ckt = Circuit()
ckt.connect(wg1.o("o1"), wg2.o("o0"))

We can also rename pins for semantic clarity. Also, anytime we specify a model without specifying a port, the first unconnected port is used.

In [13]:
wg1 = siepic.Waveguide(length=2500, loss=3)
wg2 = siepic.Waveguide(length=7500, loss=3)

# Pass in a list of strings to rename the ports of component1 in the same order 
# as they're defined in the model
wg1.rename_oports(["input", "output"])

ckt = Circuit()
# Use the next unconnected port of component2
ckt.connect(wg1.o("output"), wg2)

Here, we do the same as the previous example, except that we rename the two pins of ``component1`` to "input" and
"output", and then connect "output" to ``component2``. We do not need to explicitly specify "o1" for ``component2``,
since that is the first unconnected pin.

With this connection, we now have a rudimentary circuit to run simulations on.

## Simulation

```{eval-rst}
:py:mod:`simphony.simulators` provides a collection of simulators that connect to an input and output pin on a
circuit, then perform a subnetwork growth algorithm (a series of matrix operations). The results show us what output
light comes out of the circuit for given inputs of light.
```

<!-- The simulation process modifies pins and components, so
simulators actually copy the circuit they are passed in order to preserve the original circuit. -->

Let's run a simple sweep simulation on the circuit we have created:

In [14]:
from simphony.simulation import ClassicalSim

# Create a simulation and add a laser and detector
sim = ClassicalSim(ckt, wl=1.55)
laser = sim.add_laser(ports=ckt.o("input"), power=1.0)
detector = sim.add_detector(ports=ckt.o(1))

# Run the simulation
result = sim.run()

# Since the total wg length is 1 cm and the loss is 3 dB/cm, the power should be 50%.
print(f"Power transmission: {abs(result.output[0, 0])**2}")

Power transmission: 0.5011872053146362


/home/sequoia/git/simphony/simphony/simulation/simdevices.py:66: UserWarning: Responsivity is not yet implemented, so it is ignored.
  warnings.warn("Responsivity is not yet implemented, so it is ignored.")
/home/sequoia/git/simphony/simphony/connect.py:95: UserWarning: Explicitly requested dtype complex_ requested in zeros is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  C = jnp.zeros((nf, nC, nC), dtype="complex_")
/home/sequoia/git/simphony/simphony/simulation/classical.py:140: UserWarning: Explicitly requested dtype <class 'jax.numpy.complex128'> requested in zeros is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for mor

We instantiated up our simulator with our circuit, adding a laser input to the "input" port of ``wg1`` and placing a detector on
``wg2``. Our sweep simulation passed input light on a range of wavelengths from 1.5 microns to
1.6 microns, and now ``result`` contains what frequencies came out of our circuit. We can use these results however we like.

In order to view the results, we can use the ``matplotlib`` package to graph our output, but that will be demonstrated
in following tutorials. For this tutorial, we're done!